### Loading the models trained on Ejecta isosurface representation and saving the predicted images

In [ ]:
import os
import glob
import torch
import numpy as np
from PIL import Image
from model import models
from PIL import ImageDraw
from data import data_utils
from matplotlib import pyplot as plt

In [ ]:
# global params
width = 512
height = 512
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# volume params
center = torch.Tensor([0.0, 0.0, 0.0])
radius = 1.0
vol_params = ("sphere", center, radius)

In [ ]:
# load saved model
input_ch = 518
points_type = 'cartesian'
viewdir_type = 'cartesian'
out_dir = '/home/parika/WorkingDir/Thesis/Code/dvr/outputs/ejecta/iso/AO/21_views/gauss32.0_256_ssim+l1_withSigmoid/'
writer_path = os.path.join(out_dir,'logs')
model_path = os.path.join(out_dir,'models/best_model_74.pth')

# create the model
model = models.ConvNet(input_ch=input_ch, output_ch=1).to(device)

checkpoint = torch.load(model_path, map_location=device)
input_map = checkpoint['input_map']
model.load_state_dict(checkpoint['state_dict'])
model.eval()

In [ ]:
data_dir = '/home/parika/WorkingDir/Thesis/Data/ejecta/iso/val/'
vis_dir = os.path.join(out_dir + 'vis/val/')

with torch.no_grad():
    for cam_file in glob.glob(data_dir+"*.json"):
        fid =cam_file[cam_file.rfind('_')+1:cam_file.rfind('.')]
        gt_file = data_dir + "ao_" + fid + ".bin"

        gt = np.fromfile(gt_file, dtype=np.float32)
        gt = np.reshape(gt,(height, width))
        gt = torch.Tensor(gt).to(device)

        points, viewdirs, _ = data_utils.get_input_data(cam_file, vol_params, points_type, viewdir_type)
        gt = gt[50:500,50:500]
        points = points[:,50:500,50:500]
        viewdirs = viewdirs[:,50:500,50:500]

        mask = torch.ones(size=gt.shape,dtype=torch.bool,device=gt.device)

        input = torch.cat((points, viewdirs),dim=0).to(device)
        input = data_utils.input_mapping(input, input_map, True, True, "cartesian","ConvNet")
        input = input.unsqueeze_(0)
        ao = model(input)[0,0]

        ao = torch.clamp(ao.detach().cpu(), min=0.0, max=1.0)
        gt_img_file = vis_dir + "gt_" + fid + ".png"
        pred_img_file = vis_dir + "pred_" + fid + ".png"

        gt = Image.fromarray((gt * 255.0).numpy().astype(np.uint8))
        ao = Image.fromarray((ao * 255.0).numpy().astype(np.uint8))
        gt.save(gt_img_file)
        ao.save(pred_img_file)

### Create the zoomed in images for AO predictions and ground truth

In [ ]:
img_file = '/home/goel/Downloads/ssim/img1.png'
created_img = '/home/goel/Downloads/ssim/img1_cropped.png'
gt_img = Image.open(img_file)
gt_img = gt_img.crop((80,80,480,480))
gt_img.load()

# borders_crop = (210,210,280,280)
# borders_crop = (220,70,290,140)
borders_crop = (180, 90, 250, 160)
# borders_crop = (30,190,100,260)
# borders_crop = (150, 90, 220, 160)
gt_crop = gt_img.crop(borders_crop)
print("Size of the crop taken: ", gt_crop.size)

height, width = gt_crop.size
rescaled_size=(height*3, width*3)
gt_crop_scaled =  gt_crop.resize(rescaled_size)
print("Size of the scaled crop: ",gt_crop_scaled.size)

# Draw a box to specify where the crop is taken
width_of_rect = 1.0
draw = ImageDraw.Draw(gt_img)
draw.rectangle(borders_crop, outline='black', width=3)
print("Position where the crop is taken: ",borders_crop)

# Draw a box where the crop will be pasted
# pos_paste_crop = (0,0,rescaled_size[0],rescaled_size[1])
# draw.rectangle(pos_paste_crop, outline='black', width=3)
# print("Position where scaled crop is pasted",pos_paste_crop)

# pos_paste_crop = (0,0,rescaled_size[0],rescaled_size[1])
# pos_paste_crop = (gt_img.size[0]-rescaled_size[0],gt_img.size[1]-rescaled_size[1],gt_img.size[0],gt_img.size[1])
pos_paste_crop = (0,gt_img.size[1]-rescaled_size[1],rescaled_size[0],gt_img.size[1])
gt_img.paste(gt_crop_scaled,pos_paste_crop)

draw.rectangle(pos_paste_crop, outline='black', width=3)
print("Position where scaled crop is pasted",pos_paste_crop)

# Draw the lines to connect both the boxes
draw.line((pos_paste_crop[0],pos_paste_crop[1])+(borders_crop[0],borders_crop[1]),fill='black',width=3)
draw.line((pos_paste_crop[2],pos_paste_crop[3])+(borders_crop[2],borders_crop[3]),fill='black',width=3)

# gt_img.show()
gt_img.save(created_img)

In [ ]:
# separate tensorboard ao images into individual images
img_file = '/home/goel/Downloads/l1+ssim.png'
img = Image.open(img_file)

first_img = img.crop((0,0,512,512))
first_img.save('/home/goel/Downloads/img1.png')

second_img = img.crop((513,0,1024,512))
second_img.save('/home/goel/Downloads/img2.png')